In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('DS_Sample_1.csv')
df.shape

(1000, 18)

# 전처리

## 단계1

In [4]:
df[['CREDIT_LIMIT', 'MINIMUM_PAYMENTS']] = df[['CREDIT_LIMIT', 'MINIMUM_PAYMENTS']].apply(lambda x: x.fillna(x.mean()))

In [5]:
df['MINIMUM_PAYMENTS'].isna().sum()

0

In [6]:
card1 = df.copy()

# 1.

In [14]:
card1.groupby('TENURE')[['BALANCE', 'CREDIT_LIMIT']].corr()[::2]['CREDIT_LIMIT'].max().round(2)

0.95

# 2.

## 단계1

In [15]:
df2 = card1.copy()

In [20]:
cols = df2.columns[1:]
cols

Index(['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOFF_PURCHASES',
       'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE', 'PURCHASES_FREQUENCY',
       'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY',
       'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES_TRX',
       'CREDIT_LIMIT', 'PAYMENTS', 'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT',
       'TENURE'],
      dtype='object')

In [21]:
from sklearn.preprocessing import StandardScaler

In [22]:
scaler = StandardScaler().fit(X=df2[cols])

In [25]:
df_scaled = pd.DataFrame(scaler.transform(df2[cols]), columns=cols)

## 단계2

In [27]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [32]:
result = pd.DataFrame()

for i in [2, 3, 4, 5]:
    model = KMeans(n_clusters=i, random_state=1234).fit(df_scaled)
    cluster = model.predict(df_scaled)
    score = silhouette_score(df_scaled, cluster)
    
    result = result.append({'n_cluster': i, 'score': score}, ignore_index=True)

In [37]:
result[result['score'] == result['score'].max()]

,n_cluster,score
0,2.0,0.307528


## 단계3

In [39]:
model = KMeans(n_clusters=2, random_state=1234).fit(df_scaled)
cluster = model.predict(df_scaled)

In [43]:
df2['cluster'] = cluster

In [46]:
df2.groupby('cluster')['ONEOFF_PURCHASES'].mean().max().round(2)

3946.19

# 3.

In [47]:
card1.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001.0,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0.0,2.0,1000.0,201.802084,139.509787,0.000000,12.0
1,10002.0,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4.0,0.0,7000.0,4103.032597,1072.340217,0.222222,12.0
2,10003.0,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,12.0,7500.0,622.066742,627.284787,0.000000,12.0
3,10004.0,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1.0,1.0,7500.0,0.000000,1297.116322,0.000000,12.0
4,10005.0,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0.0,1.0,1200.0,678.334763,244.791237,0.000000,12.0


## 단계1

In [49]:
train = card1[card1['CUST_ID'] % 4 != 0]
test = card1[card1['CUST_ID'] % 4 == 0]

## 단계2

In [51]:
from sklearn.tree import DecisionTreeRegressor

In [57]:
train.columns
cols = ['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']
len(cols)

16

In [58]:
model = DecisionTreeRegressor(random_state=1234).fit(X=train[cols], y=train['ONEOFF_PURCHASES'])

## 단계3

In [60]:
pred = model.predict(X=test[cols])

In [61]:
from sklearn.metrics import mean_squared_error

In [63]:
round(mean_squared_error(y_true=test['ONEOFF_PURCHASES'], y_pred=pred) ** 0.5, 1)

2383.8